In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout,Flatten
from keras.optimizers import SGD,Adam
import matplotlib.pyplot as plt
from keras import backend as K
from mpl_toolkits.mplot3d import Axes3D
import math
import tensorflow_probability as tfp

Using TensorFlow backend.


In [2]:
seed  = 1
tf.random.set_seed(seed)
np.random.seed(seed+1)

# Tools

In [3]:
def z_score(df,i):
    df_std = df.copy()
    # apply the z-score method
    for column in df_std.columns[i:]:
        df_std[column] = (df_std[column] - df_std[column].mean()) / df_std[column].std()
    return df_std
def correlation_coefficient(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = K.mean(x)
    my = K.mean(y)
    xm, ym = x-mx, y-my
    r_num = K.sum(tf.multiply(xm,ym))
    r_den = K.sqrt(tf.multiply(K.sum(K.square(xm)), K.sum(K.square(ym))))
    r = r_num / r_den
    r = K.maximum(K.minimum(r, 1.0), -1.0)
    #return 1-K.square(r)?
    return r

def correlation(y_true,y_pred):
    corr = tfp.stats.correlation(y_true, y_pred, sample_axis=0, event_axis=None)
    return corr

def Remove(duplicate): 
    final_list = [] 
    for num in duplicate: 
        if num not in final_list: 
            final_list.append(num) 
    return final_list

# Model Modules

In [4]:
class Linear(keras.layers.Layer):
    def __init__(self, units, input_dim):
        super(Linear, self).__init__()
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value=w_init(shape=(input_dim, units), dtype="float32"),
            trainable=True,
        )
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(
            initial_value=b_init(shape=(units,), dtype="float32"), trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

In [5]:
def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    return pos * angle_rates


def positional_encoding(position, d_model):
    angle_rads = get_angles( np.arange(position)[:, np.newaxis], np.arange(d_model)[np.newaxis, :], d_model)
    
    # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    
    # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads[np.newaxis, ...]
    return tf.cast(pos_encoding, dtype=tf.float32)

In [12]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, causal=False, dropout=0.0):
    super(MultiHeadAttention, self).__init__()

    assert d_model % num_heads == 0
    depth = d_model // num_heads

    self.w_query = tf.keras.layers.Dense(d_model)
    self.split_reshape_query = tf.keras.layers.Reshape((-1,num_heads,depth))  
    self.split_permute_query = tf.keras.layers.Permute((2,1,3))      

    self.w_value = tf.keras.layers.Dense(d_model)
    self.split_reshape_value = tf.keras.layers.Reshape((-1,num_heads,depth))
    self.split_permute_value = tf.keras.layers.Permute((2,1,3))

    self.w_key = tf.keras.layers.Dense(d_model)
    self.split_reshape_key = tf.keras.layers.Reshape((-1,num_heads,depth))
    self.split_permute_key = tf.keras.layers.Permute((2,1,3))

    self.attention = tf.keras.layers.Attention(causal=causal)  #dropout=dropout
    self.join_permute_attention = tf.keras.layers.Permute((2,1,3))
    self.join_reshape_attention = tf.keras.layers.Reshape((-1,d_model))

    self.dense = tf.keras.layers.Dense(d_model)

  def call(self, inputs, mask=None, training=None):
    q = inputs[0]
    v = inputs[1]
    k = inputs[2] if len(inputs) > 2 else v

    query = self.w_query(q)
    query = self.split_reshape_query(query)    
    query = self.split_permute_query(query)                 

    value = self.w_value(v)
    value = self.split_reshape_value(value)
    value = self.split_permute_value(value)

    key = self.w_key(k)
    key = self.split_reshape_key(key)
    key = self.split_permute_key(key)

    if mask is not None:
      if mask[0] is not None:
        mask[0] = tf.keras.layers.Reshape((-1,1))(mask[0])
        mask[0] = tf.keras.layers.Permute((2,1))(mask[0])
      if mask[1] is not None:
        mask[1] = tf.keras.layers.Reshape((-1,1))(mask[1])
        mask[1] = tf.keras.layers.Permute((2,1))(mask[1])

    attention = self.attention([query, value, key], mask=mask)
    attention = self.join_permute_attention(attention)
    attention = self.join_reshape_attention(attention)

    x = self.dense(attention)

    return x

# Encoder

In [7]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, input_vocab_size, num_layers = 1, d_model = 24, num_heads = 1, dff = 32, maximum_position_encoding = 10000, dropout = 0.0):
    super(Encoder, self).__init__()

    self.d_model = d_model

    #self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model, mask_zero=True)
    self.embedding =Linear(d_model, input_vocab_size)
    
    self.pos = positional_encoding(maximum_position_encoding, d_model)

    self.encoder_layers = [ EncoderLayer(d_model = d_model, num_heads = num_heads, dff = dff, dropout = 0.0) for _ in range(num_layers)]

    #self.dropout = tf.keras.layers.Dropout(dropout)

  def call(self, inputs, mask=None, training=None):
    x = self.embedding(inputs)
    x=tf.reshape(x,[-1,1,d_model])
    # positional encoding
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32)) 
    x += self.pos[: , :tf.shape(x)[1], :]

    #x = self.dropout(x, training=training)

    #Encoder layer
    #embedding_mask = self.embedding.compute_mask(inputs)
    for encoder_layer in self.encoder_layers:
      x = encoder_layer(x, mask = None)

    return x

  def compute_mask(self, inputs, mask=None):
    return self.embedding.compute_mask(inputs)

In [8]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self,  d_model, num_heads, dff, dropout):
    super(EncoderLayer, self).__init__()

    self.multi_head_attention =  MultiHeadAttention(d_model, num_heads)
    #self.dropout_attention = tf.keras.layers.Dropout(dropout)
    self.add_attention = tf.keras.layers.Add()
    self.layer_norm_attention = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dense1 = tf.keras.layers.Dense(dff, activation='relu')
    self.dense2 = tf.keras.layers.Dense(d_model)
    #self.dropout_dense = tf.keras.layers.Dropout(dropout)
    self.add_dense = tf.keras.layers.Add()
    self.layer_norm_dense = tf.keras.layers.LayerNormalization(epsilon=1e-6)

  def call(self, inputs, mask=None, training=None):
    # print(mask)
    attention = self.multi_head_attention([inputs,inputs,inputs], mask = [mask,mask])
    #attention = self.dropout_attention(attention, training = training)
    x = self.add_attention([inputs , attention])
    x = self.layer_norm_attention(x)

    ## Feed Forward
    dense = self.dense1(x)
    dense = self.dense2(dense)
    #dense = self.dropout_dense(dense, training = training)
    x = self.add_dense([x , dense])
    x = self.layer_norm_dense(x)

    return x

# Decoder

In [9]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, target_vocab_size, num_layers, d_model, num_heads, dff, maximum_position_encoding = 10000, dropout = 0.0):
    super(Decoder, self).__init__()

    self.d_model = d_model

    #self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model, mask_zero=True)
    self.embedding =Linear(d_model, target_vocab_size)
    
    self.pos = positional_encoding(maximum_position_encoding, d_model)

    self.decoder_layers = [ DecoderLayer(d_model = d_model, num_heads = num_heads, dff = dff, dropout = 0.0)  for _ in range(num_layers)]

    #self.dropout = tf.keras.layers.Dropout(dropout)

  def call(self, inputs, mask=None, training=None):
    x = self.embedding(inputs[0])
    x=tf.reshape(x,[-1,1,d_model])
    
    # positional encoding
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos[: , :tf.shape(x)[1], :]
    #x = self.dropout(x, training=training)

    #Decoder layer
    #embedding_mask = self.embedding.compute_mask(inputs[0])
    for decoder_layer in self.decoder_layers:
      x = decoder_layer([x,inputs[1]], mask = [None, None])

    return x

  # Comment this out if you want to use the masked_loss()
  def compute_mask(self, inputs, mask=None):
    return self.embedding.compute_mask(inputs[0])

In [10]:
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self,  d_model, num_heads, dff, dropout = 0.0):
    super(DecoderLayer, self).__init__()

    self.multi_head_attention1 =  MultiHeadAttention(d_model, num_heads, causal = True)
    #self.dropout_attention1 = tf.keras.layers.Dropout(dropout)
    self.add_attention1 = tf.keras.layers.Add()
    self.layer_norm_attention1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.multi_head_attention2 =  MultiHeadAttention(d_model, num_heads)
    #self.dropout_attention2 = tf.keras.layers.Dropout(dropout)
    self.add_attention2 = tf.keras.layers.Add()
    self.layer_norm_attention2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)


    self.dense1 = tf.keras.layers.Dense(dff, activation='relu')
    self.dense2 = tf.keras.layers.Dense(d_model)
    #self.dropout_dense = tf.keras.layers.Dropout(dropout)
    self.add_dense = tf.keras.layers.Add()
    self.layer_norm_dense = tf.keras.layers.LayerNormalization(epsilon=1e-6)

  def call(self, inputs, mask=None, training=None):
    # print(mask)
    attention = self.multi_head_attention1([inputs[0],inputs[0],inputs[0]], mask = [mask[0],mask[0]])
    #attention = self.dropout_attention1(attention, training = training)
    x = self.add_attention1([inputs[0] , attention])
    x = self.layer_norm_attention1(x)
    
    attention = self.multi_head_attention2([x, inputs[1],inputs[1]], mask = [mask[0],mask[1]])
    #attention = self.dropout_attention2(attention, training = training)
    x = self.add_attention1([x , attention])
    x = self.layer_norm_attention1(x)


    ## Feed Forward
    dense = self.dense1(x)
    dense = self.dense2(dense)
    #dense = self.dropout_dense(dense, training = training)
    x = self.add_dense([x , dense])
    x = self.layer_norm_dense(x)

    return x

# Model

In [14]:
# Hyperparameters
num_layers = 1
d_model = 24
dff = 32
num_heads = 1
dropout_rate = 0

# Size of input vocab plus start and end tokens
input_vocab_size = 12
target_vocab_size = 2

input = tf.keras.layers.Input(shape=(None,))
target = tf.keras.layers.Input(shape=(None,))

encoder = Encoder(input_vocab_size, num_layers = num_layers, d_model = d_model, num_heads = num_heads, dff = dff, dropout = dropout_rate)
decoder = Decoder(target_vocab_size, num_layers = num_layers, d_model = d_model, num_heads = num_heads, dff = dff, dropout = dropout_rate)

x = encoder(input)
x = decoder([target, x] , mask = encoder.compute_mask(input))
x = tf.keras.layers.Dense(target_vocab_size)(x)

base_model = tf.keras.models.Model(inputs=[input, target], outputs=x)
sgd = tf.keras.optimizers.SGD(lr = 0.05)
base_model.compile(loss='mse', optimizer=sgd,metrics=[correlation_coefficient])
base_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
encoder_2 (Encoder)             (None, None, 24)     4400        input_5[0][0]                    
__________________________________________________________________________________________________
decoder_1 (Decoder)             (None, None, 24)     6560        input_6[0][0]                    
                                                                 encoder_2[0][0]            

# DataSets

In [15]:
MyData=pd.read_excel('MyData.xlsx', sheet_name="Train1m2")
MyData.columns

Index(['Day', 'Symbol', 'Sector', 'Return1w', 'Return2w', 'Return3w', 'Return',
       'PER', 'EV_EBITDA', 'PC1M', 'FCFYield', 'SalesYield', 'PBRatio',
       'WCRYield', 'FCFPRatio1YearGrowth', 'TotalYield',
       'EVToTotalAssets1YearGrowth', 'Vol', 'Mean_Rev'],
      dtype='object')

In [16]:
Remove(list(MyData['Day']))

[21, 41, 61, 81, 101, 121, 141, 161]

In [24]:
N=161
elem="FinancialServices"
#x_Train=MyData[(MyData['Day']<N)&(MyData['Sector']==elem)][MyData.columns[7:]]
x_Train=MyData[MyData['Day']<N][MyData.columns[7:]]
x_Train=z_score(x_Train,0)
x_Train=x_Train.values
#y_Train=MyData[(MyData['Day']<N)&(MyData['Sector']==elem)][["Return1w","Return2w","Return3w","Return"]].values
y_Train=MyData[MyData['Day']<N][["Return1w","Return2w","Return"]].values
#y_Train=np.c_[np.zeros(y_Train.shape[0]),y_Train]

#x_Test=MyData[(MyData['Day']==N)&(MyData['Sector']==elem)][MyData.columns[7:]]
x_Test=MyData[MyData['Day']==N][MyData.columns[7:]]
x_Test=z_score(x_Test,0)
x_Test=x_Test.values
#y_Test=MyData[(MyData['Day']==N)&(MyData['Sector']==elem)][["Return1w","Return2w","Return3w","Return"]].values
y_Test=MyData[MyData['Day']==N][["Return1w","Return2w","Return"]].values
#y_Test=np.c_[np.zeros(y_Test.shape[0]),y_Test]

In [25]:
(x_Train.shape,y_Train.shape,x_Test.shape,y_Test.shape)

((3577, 12), (3577, 3), (511, 12), (511, 3))

In [469]:
#y_Train=y_Train.reshape(-1,1,5)
#y_Test=y_Test.reshape(-1,1,5)
#y_Test.shape

# Train Model

In [23]:
base_model.fit(x = [x_Train, y_Train[:, :-1]], y = y_Train[:, 1:],
               batch_size=32,epochs=150,validation_data=([x_Test, y_Test[:, :-1]], y_Test[:, 1:]))

Train on 3066 samples, validate on 511 samples
Epoch 1/150
3066/3066 [==============================] - 1s 184us/sample - loss: 0.0065 - correlation_coefficient: 0.6140 - val_loss: 0.0033 - val_correlation_coefficient: 0.6329
Epoch 2/150
3066/3066 [==============================] - 1s 170us/sample - loss: 0.0065 - correlation_coefficient: 0.6734 - val_loss: 0.0034 - val_correlation_coefficient: 0.6343
Epoch 3/150
3066/3066 [==============================] - 1s 171us/sample - loss: 0.0065 - correlation_coefficient: 0.6841 - val_loss: 0.0035 - val_correlation_coefficient: -0.6281
Epoch 4/150
3066/3066 [==============================] - 1s 172us/sample - loss: 0.0065 - correlation_coefficient: 0.6341 - val_loss: 0.0035 - val_correlation_coefficient: 0.6260
Epoch 5/150
3066/3066 [==============================] - 1s 180us/sample - loss: 0.0066 - correlation_coefficient: 0.4649 - val_loss: 0.0032 - val_correlation_coefficient: 0.6333
Epoch 6/150
3066/3066 [==============================] - 

3066/3066 [==============================] - 1s 176us/sample - loss: 0.0064 - correlation_coefficient: 0.6168 - val_loss: 0.0034 - val_correlation_coefficient: 0.6273
Epoch 92/150
3066/3066 [==============================] - 1s 185us/sample - loss: 0.0065 - correlation_coefficient: 0.6240 - val_loss: 0.0033 - val_correlation_coefficient: 0.6347
Epoch 93/150
3066/3066 [==============================] - 1s 177us/sample - loss: 0.0065 - correlation_coefficient: 0.6449 - val_loss: 0.0033 - val_correlation_coefficient: 0.5926
Epoch 94/150
3066/3066 [==============================] - 1s 186us/sample - loss: 0.0065 - correlation_coefficient: 0.6460 - val_loss: 0.0032 - val_correlation_coefficient: 0.6325
Epoch 95/150
3066/3066 [==============================] - 1s 181us/sample - loss: 0.0065 - correlation_coefficient: 0.7697 - val_loss: 0.0034 - val_correlation_coefficient: 0.6333
Epoch 96/150
3066/3066 [==============================] - 1s 184us/sample - loss: 0.0065 - correlation_coefficien

# Prediction